# 🔥 Demostración: Sistema de Predicción de Incendios Forestales

Este notebook demuestra el uso completo del sistema de predicción de incendios forestales, desde la carga de datos hasta la realización de predicciones.

**Autor:** Herwig  
**Fecha:** 2024  
**Tecnologías:** Python, Scikit-learn, Pandas

## 1. Configuración Inicial

In [ ]:
# Importar librerías necesarias
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Agregar el directorio src al path
sys.path.append(str(Path.cwd().parent / 'src'))

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Librerías importadas correctamente")
print(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Cargar el Sistema de Predicción

In [ ]:
from models.predict import FirePredictionSystem, create_example_input

# Rutas de los modelos
model_path = '../models/random_forest_model.pkl'
scaler_path = '../models/scaler.pkl'

# Verificar existencia
if not Path(model_path).exists():
    print("⚠️ Modelo no encontrado. Ejecuta train_random_forest.py primero.")
else:
    # Inicializar sistema
    predictor = FirePredictionSystem(
        model_path=model_path,
        scaler_path=scaler_path if Path(scaler_path).exists() else None
    )
    print("✓ Sistema de predicción cargado correctamente")

## 3. Predicción Individual

Vamos a realizar una predicción con datos de ejemplo.

In [ ]:
# Crear datos de ejemplo
ejemplo = create_example_input()

# Mostrar datos de entrada
print("📊 DATOS DE ENTRADA:")
print("=" * 50)
for key, value in ejemplo.items():
    print(f"{key:20s}: {value}")
print("=" * 50)

In [ ]:
# Realizar predicción
resultado = predictor.predict_with_details(ejemplo)

# Mostrar resultado
print("\n🎯 RESULTADO DE PREDICCIÓN:")
print("=" * 50)
print(f"Predicción: {resultado['prediction_label']}")
print(f"Nivel de Riesgo: {resultado['risk_level']}")
if resultado['confidence']:
    print(f"Confianza: {resultado['confidence']:.2%}")
print(f"Timestamp: {resultado['timestamp']}")
print("=" * 50)

## 4. Visualización de Probabilidades

In [ ]:
if resultado['probabilities'] is not None:
    # Preparar datos
    clases = ['Sin Riesgo', 'Riesgo Bajo', 'Riesgo Medio', 
              'Riesgo Alto', 'Riesgo Crítico'][:len(resultado['probabilities'])]
    probs = resultado['probabilities']
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['green', 'yellowgreen', 'yellow', 'orange', 'red'][:len(probs)]
    bars = ax.bar(clases, probs, color=colors, alpha=0.7, edgecolor='black')
    
    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1%}',
                ha='center', va='bottom', fontweight='bold')
    
    ax.set_ylabel('Probabilidad', fontsize=12, fontweight='bold')
    ax.set_xlabel('Nivel de Riesgo', fontsize=12, fontweight='bold')
    ax.set_title('Distribución de Probabilidades', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 1.1])
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No hay probabilidades disponibles para este modelo")

## 5. Predicción por Lotes

Ahora vamos a realizar predicciones múltiples en un DataFrame.

In [ ]:
# Generar datos sintéticos para múltiples predicciones
np.random.seed(42)
n_samples = 20

datos_lote = pd.DataFrame({
    'temperatura': np.random.uniform(15, 40, n_samples),
    'humedad': np.random.uniform(20, 80, n_samples),
    'velocidad_viento': np.random.uniform(5, 30, n_samples),
    'precipitacion': np.random.uniform(0, 20, n_samples),
    'indice_sequedad': np.random.uniform(20, 80, n_samples),
    'vegetacion_seca': np.random.uniform(0.2, 0.9, n_samples),
    'mes': np.random.randint(1, 13, n_samples),
    'dia_semana': np.random.randint(0, 7, n_samples)
})

print(f"✓ Generados {n_samples} casos de prueba")
print("\nPrimeras 5 observaciones:")
datos_lote.head()

In [ ]:
# Realizar predicciones en lote
resultados_lote = predictor.batch_predict(datos_lote)

print(f"✓ Predicciones completadas: {len(resultados_lote)} casos")
print("\nPrimeras 5 predicciones:")
resultados_lote.head()

## 6. Análisis de Resultados

In [ ]:
# Estadísticas de las predicciones
print("📊 ESTADÍSTICAS DE PREDICCIONES:")
print("=" * 50)
print(f"Total de predicciones: {len(resultados_lote)}")
print("\nDistribución por nivel de riesgo:")
print(resultados_lote['etiqueta_prediccion'].value_counts())
print("\nPorcentajes:")
print(resultados_lote['etiqueta_prediccion'].value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))
print("=" * 50)

In [ ]:
# Visualización de la distribución
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras
pred_counts = resultados_lote['etiqueta_prediccion'].value_counts()
colors_map = {
    'Sin Riesgo': 'green',
    'Riesgo Bajo': 'yellowgreen',
    'Riesgo Medio': 'yellow',
    'Riesgo Alto': 'orange',
    'Riesgo Crítico': 'red'
}
bar_colors = [colors_map.get(x, 'gray') for x in pred_counts.index]

axes[0].bar(range(len(pred_counts)), pred_counts.values, color=bar_colors, alpha=0.7, edgecolor='black')
axes[0].set_xticks(range(len(pred_counts)))
axes[0].set_xticklabels(pred_counts.index, rotation=45, ha='right')
axes[0].set_ylabel('Frecuencia', fontweight='bold')
axes[0].set_title('Distribución de Predicciones', fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Gráfico de torta
axes[1].pie(pred_counts.values, labels=pred_counts.index, autopct='%1.1f%%',
            colors=bar_colors, startangle=90)
axes[1].set_title('Proporción de Niveles de Riesgo', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Análisis de Relaciones

Exploremos cómo las variables meteorológicas se relacionan con el riesgo predicho.

In [ ]:
# Boxplot de temperatura por nivel de riesgo
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Temperatura
resultados_lote.boxplot(column='temperatura', by='etiqueta_prediccion', ax=axes[0, 0])
axes[0, 0].set_title('Temperatura por Nivel de Riesgo')
axes[0, 0].set_xlabel('Nivel de Riesgo')
axes[0, 0].set_ylabel('Temperatura (°C)')

# Humedad
resultados_lote.boxplot(column='humedad', by='etiqueta_prediccion', ax=axes[0, 1])
axes[0, 1].set_title('Humedad por Nivel de Riesgo')
axes[0, 1].set_xlabel('Nivel de Riesgo')
axes[0, 1].set_ylabel('Humedad (%)')

# Velocidad del viento
resultados_lote.boxplot(column='velocidad_viento', by='etiqueta_prediccion', ax=axes[1, 0])
axes[1, 0].set_title('Velocidad del Viento por Nivel de Riesgo')
axes[1, 0].set_xlabel('Nivel de Riesgo')
axes[1, 0].set_ylabel('Velocidad (km/h)')

# Índice de sequedad
resultados_lote.boxplot(column='indice_sequedad', by='etiqueta_prediccion', ax=axes[1, 1])
axes[1, 1].set_title('Índice de Sequedad por Nivel de Riesgo')
axes[1, 1].set_xlabel('Nivel de Riesgo')
axes[1, 1].set_ylabel('Índice de Sequedad')

plt.suptitle('')  # Remover el título general automático
plt.tight_layout()
plt.show()

## 8. Estadísticas del Sistema

In [ ]:
# Obtener estadísticas del sistema
stats = predictor.get_statistics()

print("📊 ESTADÍSTICAS DEL SISTEMA:")
print("=" * 50)
print(f"Total de predicciones realizadas: {stats['total_predictions']}")

if stats['prediction_distribution']:
    print("\nDistribución de predicciones:")
    for pred, count in stats['prediction_distribution'].items():
        print(f"  {pred}: {count} veces")

if stats['most_common_prediction'] is not None:
    print(f"\nPredicción más común: {stats['most_common_prediction']}")

print("=" * 50)

## 9. Casos de Uso Práctico

### Escenario 1: Condiciones de alto riesgo

In [ ]:
# Simular condiciones de alto riesgo
alto_riesgo = {
    'temperatura': 38.0,          # Alta temperatura
    'humedad': 25.0,              # Baja humedad
    'velocidad_viento': 35.0,     # Viento fuerte
    'precipitacion': 0.0,         # Sin lluvia
    'indice_sequedad': 85.0,      # Alta sequedad
    'vegetacion_seca': 0.9,       # Mucha vegetación seca
    'mes': 7,                     # Julio (verano)
    'dia_semana': 6               # Domingo
}

resultado_alto = predictor.predict_with_details(alto_riesgo)

print("⚠️ ESCENARIO DE ALTO RIESGO:")
print("=" * 50)
print(f"Predicción: {resultado_alto['prediction_label']}")
print(f"Nivel de Riesgo: {resultado_alto['risk_level']}")
if resultado_alto['confidence']:
    print(f"Confianza: {resultado_alto['confidence']:.2%}")
print("=" * 50)

### Escenario 2: Condiciones favorables

In [ ]:
# Simular condiciones favorables
bajo_riesgo = {
    'temperatura': 18.0,          # Temperatura moderada
    'humedad': 70.0,              # Alta humedad
    'velocidad_viento': 8.0,      # Viento suave
    'precipitacion': 15.0,        # Lluvia reciente
    'indice_sequedad': 25.0,      # Baja sequedad
    'vegetacion_seca': 0.2,       # Poca vegetación seca
    'mes': 11,                    # Noviembre (otoño)
    'dia_semana': 2               # Miércoles
}

resultado_bajo = predictor.predict_with_details(bajo_riesgo)

print("✅ ESCENARIO DE BAJO RIESGO:")
print("=" * 50)
print(f"Predicción: {resultado_bajo['prediction_label']}")
print(f"Nivel de Riesgo: {resultado_bajo['risk_level']}")
if resultado_bajo['confidence']:
    print(f"Confianza: {resultado_bajo['confidence']:.2%}")
print("=" * 50)

## 10. Conclusiones

Este notebook ha demostrado:

1. **Carga y configuración** del sistema de predicción
2. **Predicciones individuales** con datos de ejemplo
3. **Predicciones por lotes** para análisis masivo
4. **Visualización de resultados** con gráficos informativos
5. **Análisis de relaciones** entre variables meteorológicas y riesgo
6. **Casos de uso prácticos** con escenarios reales

El sistema está listo para su uso en producción y puede integrarse con:
- APIs REST para aplicaciones web
- Dashboards interactivos
- Sistemas de monitoreo en tiempo real
- Alertas automáticas

---

**Próximos pasos:**
- Integrar con datos meteorológicos reales
- Implementar sistema de alertas
- Desplegar en producción
- Reentrenar periódicamente con nuevos datos